In [2]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import cv2
import numpy as np
from keras import models
import cv2
import numpy as np
import re
from cv2 import dnn_superres
from time import sleep

In [3]:
main_folder_path = os.path.dirname(os.getcwd())

In [4]:
config_path = os.path.join(main_folder_path,'config\\pipeline.config')
model_character_path = r'D:\Code_school_nam3ki2\TestModel\model\classfication_character\model_license_plate_v9.h5'
label_path = os.path.join(main_folder_path, 'config\\label_map.pbtxt')
checkpoint_path = os.path.join(main_folder_path, 'model\\detect_liscense_plate')
test_folder_path = r'D:\car_long'
result_folder_path = os.path.join(main_folder_path,'images_result')

In [7]:
category_index = label_map_util.create_category_index_from_labelmap(label_path)
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(config_path)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
model_checkpoint = ckpt.restore(os.path.join(checkpoint_path, 'ckpt-11')).expect_partial()

model_character = models.load_model(model_character_path)

In [8]:
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [9]:
def pre_process(img, W):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bfilter = cv2.bilateralFilter(img_gray, 11, 17, 17) #Noise reduction
    # blur = cv2.GaussianBlur(img_gray, (3, 3), 0)
    binary = cv2.adaptiveThreshold(bfilter, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, int(W/20) if int(W/20)%2 !=0  else int(W/20)+1, 15)
    return binary

In [10]:
def predict_license_plate(img):
    image_np = np.array(img)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    image_np_crop = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'], 
                detections['detection_classes'] + label_id_offset, #Cộng vô để khớp với category_index
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True, #Chuấn hóa về 0 => 1
                max_boxes_to_draw=2, 
                agnostic_mode=False) #Tất cả các hộp đều được vẽ cùng màu
    return image_np_with_detections, image_np_crop, detections['detection_boxes'][0]

def get_detections(img, detections):
    img_crop = None
    ymin, xmin, ymax, xmax = detections
    height, width, _ = img.shape
    ymin = int(ymin * height)
    ymax = int(ymax * height)
    xmin = int(xmin * width)
    xmax = int(xmax * width)
    X = xmin
    Y = ymin
    W = xmax - xmin
    H = ymax - ymin
    if X != None and Y != None and W != None and H != None:
        #Cắt vùng chứa biển số xe
        img_crop = img[int(Y)-10: int(Y)+int(H) + 10, int(X)-10: int(X)+int(W)+10] if int(Y)-5 > 0 and int(X)-5 > 0 else img[int(Y): int(Y)+int(H), int(X): int(X)+int(W)]
        cv2.imwrite(os.path.join(result_folder_path, "cropped_image.jpg"), img_crop)
        img_crop = cv2.imread(os.path.join(result_folder_path, "cropped_image.jpg"))
        #Loại bỏ nhiễu 
        img_crop = cv2.fastNlMeansDenoisingColored(img_crop, None, 10, 10, 7, 21)
    return img_crop, X,Y,W,H

In [11]:
def segmentation_character(binary, img, img_crop, model_character, X, Y, W, H, type = 0) :
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, connectivity=8)
    print("Num_labels:", num_labels)
    candidates = []
    bounding_rects = []
    list_character = []
    i = 0
    for label in range(1, num_labels):
        # Tạo mask chứa các pixel có nhãn cùng là label
        mask = np.zeros(binary.shape, dtype=np.uint8)
        mask[labels == label] = 255 # Các các pixel cùng nhãn giá trị 255
        # Tìm contours từ mask
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Lọc contours theo tiêu chí aspect ratio, solidity và height ratio
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = w / float(h)
            solidity = cv2.contourArea(contour) / float(w * h)
            height_ratio = h / float(binary.shape[0])


            #Loại bỏ nhiễu dựa vào aspect ratio, solidity và height ratio
            if 0.2 < aspect_ratio < 0.8 and solidity > 0.1 and 0.2 < height_ratio < 1.0:
                bounding_rects.append((x, y, w, h))
                # Trích xuất ký tự
                character = img_crop[y-3: y + h+3, x-int(h*3/10-w/2):x + w+int(h*3/10-w/2)]
                candidates.append((x, y, character))

    if candidates[-1][1]/float(candidates[0][1]) > 2:  
        list_character_first = []
        list_character_second = []          
        first_lines = [item for item in candidates if (candidates[-1][1]/float(item[1])) >= 2.0 ]
        second_lines = [item for item in candidates if (candidates[-1][1]/float(item[1])) < 2.0 ]
        first_lines.sort(key=lambda item: item[0])
        first_lines = [item for item in first_lines]
        second_lines.sort(key=lambda item: item[0])
        second_lines = [item for item in second_lines]
        for first_line in first_lines:
                i+=1
                filename = f"region_{i}.jpg"
                save_path = os.path.join(result_folder_path, filename)
                cv2.imwrite(save_path, first_line[2])
                list_character_first.append(predict_image(save_path,model_character))

        for idx, (x, y,char)  in enumerate(first_lines):
        # Vẽ hình chữ nhật bao quanh ký tự
            cv2.rectangle(img, (x+int(X)-4, y+int(Y)-3), (x+int(X) + char.shape[1] - 6, y+int(Y) + char.shape[0] - 5), (0, 255, 0), 2)
        
        # Hiển thị ký tự lên ảnh
            cv2.putText(img, list_character_first[idx], (x+int(X)-3, y + int(Y)-3), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)   
                    
        for second_line in second_lines:
                i+=1
                filename = f"region_{i}.jpg"
                save_path = os.path.join(result_folder_path, filename)
                cv2.imwrite(save_path, second_line[2])
                list_character_second.append(predict_image(save_path,model_character))
        list_character.extend(list_character_first)
        list_character.extend(list_character_second)
        for idx, (x, y,char)  in enumerate(second_lines):
        # Vẽ hình chữ nhật bao quanh ký tự
            cv2.rectangle(img, (x+int(X)-4, y+int(Y)-3), (x+int(X) + char.shape[1] - 6, y+int(Y) + char.shape[0] - 5), (0, 255, 0), 2)
        
        # Hiển thị ký tự lên ảnh
            cv2.putText(img, list_character_second[idx], (x+int(X)-3, y + int(Y)-3), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Hiển thị ảnh với các ký tự đã nhận dạng được vẽ lên
        cv2.imshow("Detected Characters", img)
    else:
        candidates.sort(key=lambda item: item[0])
        candidates = [item for item in candidates]     
        for character in candidates:
                i+=1
                filename = f"region_{i}.jpg"
                save_path = os.path.join(result_folder_path, filename)
                cv2.imwrite(save_path, character[2])
                list_character.append(predict_image(save_path,model_character))
        for idx, (x, y,char)  in enumerate(candidates):
        # Vẽ hình chữ nhật bao quanh ký tự
            cv2.rectangle(img, (x+int(X)-4, y+int(Y)-3), (x+int(X) + char.shape[1] - 6, y+int(Y) + char.shape[0] - 5), (0, 255, 0), 2)
    
        # Hiển thị ký tự lên ảnh
            cv2.putText(img, list_character[idx], (x+int(X)-3, y + int(Y)-3), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
        # Hiển thị ảnh với các ký tự đã nhận dạng được vẽ lên
        cv2.imshow("Detected Characters", img)
    # cv2.imshow("Detected Characters", img)
    if type == 0:
        cv2.waitKey()
        cv2.destroyAllWindows()
    
    return "".join(list_character)

def predict_image(image_path, model):
    # img = improve_image_resolution(cv2.imread(image_path), "EDSR_x4.pb")
    img = cv2.imread(image_path)  
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized_img = cv2.resize(gray_img, (40,30))
    img_array = np.array(resized_img) 
    img_array = img_array/255.0
    img_input = img_array.reshape((-1, 30, 40, 1))
    pred = model.predict(img_input)
    labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
    predicted_label = labels[np.argmax(pred)]
    return predicted_label

In [12]:
def predict_image_2(img, model):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized_img = cv2.resize(gray_img, (40,30))
    img_array = np.array(resized_img) 
    img_array = img_array/255.0
    img_input = img_array.reshape((-1, 30, 40, 1))
    pred = model.predict(img_input)
    labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
    predicted_label = labels[np.argmax(pred)]
    return predicted_label

In [13]:
def image_detect(IMAGE_PATH, model_character):
    image_original = cv2.imread(IMAGE_PATH)
    image_np_with_detections, img, results = predict_license_plate(image_original)
    cv2.imshow('image_np_with_detections', image_np_with_detections)
    cv2.waitKey()
    cv2.destroyAllWindows()
    img_crop, X,Y,W,H = get_detections(img, results)
    cv2.imshow('img_crop', img_crop)
    cv2.waitKey()
    cv2.destroyAllWindows()
    binary = pre_process(img_crop, W)
    cv2.imshow('binary', binary)
    cv2.waitKey()
    cv2.destroyAllWindows()
    segmentation_character(binary, img, img_crop, model_character, X, Y, W, H)

In [14]:
def camera_detect(model_character):
    cap = cv2.VideoCapture(1)
    while cap.isOpened():
        try:
            img_crop = None
            ret, img = cap.read()
            if not ret:
                print("Không thể đọc từ camera.")
                break
            img = cv2.resize(img, (800, 600))
            cv2.imshow('img', img)
            image_np_with_detections, img, results = predict_license_plate(img)
            # cv2.imshow('image_np_with_detections', image_np_with_detections)
            img_crop, X,Y,W,H = get_detections(img, results)
            if img_crop is not None:
                binary = pre_process(img_crop, W)
                list_character = segmentation_character(binary, img, img_crop, model_character, X, Y, W, H, 1)
                print(list_character)
            else:
                print("Không tìm thấy biển số.")
            if cv2.waitKey(10) == ord('q'):
                break
        except Exception as e:
            print(f"Đã xảy ra lỗi: {e}")
    cap.release()
    cv2.destroyAllWindows()

In [15]:
def detect_line(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bfilter = cv2.bilateralFilter(img_gray, 11, 17, 17) #Noise reduction
    img_blur = cv2.GaussianBlur(bfilter, (3, 3), 0)
    # Chuyển đổi ảnh sang nhị phân
    ret, img_bin = cv2.threshold(img_blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Đảo ngược ảnh (nền đen, chữ trắng)
    img_bin = 255 - img_bin
    canny = cv2.Canny(bfilter, 50, 150)
    cv2.imshow('canny', canny)
    cv2.waitKey()
    cv2.destroyAllWindows()
    lines = cv2.HoughLinesP(canny, 1, np.pi/180, threshold=30, minLineLength=50, maxLineGap=2)
    lines_1 = cv2.HoughLinesP(img_bin, 1, np.pi/180, threshold=105, minLineLength=20, maxLineGap=10)
    return lines, lines_1

In [ ]:
list_test = os.listdir(test_folder_path)
list_test = [os.path.join(test_folder_path, image) for image in list_test if not image.endswith('.xml')]
def line_length(line):
    x1, y1, x2, y2 = line[0]
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
for img_path in list_test:
    try: 
        image_original = cv2.imread(img_path)
        image_np_with_detections, img, results = predict_license_plate(image_original)
        cv2.imshow('image_np_with_detections', image_np_with_detections)
        cv2.waitKey()
        cv2.destroyAllWindows()
        img_crop, X, Y, W, H = get_detections(img, results)
        cv2.imshow('img_crop', img_crop)
        cv2.waitKey()
        cv2.destroyAllWindows()
        # img_draw = image_original.copy()
        # img_draw_1 = image_original.copy()
        # lines, lines_1 = detect_line(image_original)
        # if lines is not None:
        #     lines = sorted(lines, key=line_length, reverse=True)[:5] if len(lines) >= 5 else lines
        #     for line in lines:
        #         x, y, w, h = line[0]
        #         cv2.line(img_draw, (x, y), (w, h), (0, 255, 0), 2)
        #     cv2.imshow('img_draw', img_draw)
        #     cv2.waitKey()
        #     cv2.destroyAllWindows()
        #     angles = []
        #     for line in lines:
        #         x1, y1, x2, y2 = line[0]
        #         angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        #         angles.append(angle)
        #     # Tính góc nghiêng trung bình
        #     angle = np.mean(angles)
        #     if np.abs(angle) < 20:
        #         # Xoay ảnh lại để chữ số thẳng đứng
        #         height, width = image_original.shape[:2]
        #         center = (width // 2, height // 2)
        #         M = cv2.getRotationMatrix2D(center, angle, 1)
        #         img_rotated = cv2.warpAffine(image_original, M, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
        #         cv2.imshow('img_rotated', img_rotated)
        #         cv2.waitKey()
        #         cv2.destroyAllWindows()
        # elif lines_1 is not None:
        #     lines_1 = sorted(lines_1, key=line_length, reverse=True)[:5] if len(lines_1) >= 5 else lines_1
        #     for line in lines_1:
        #         x, y, w, h = line[0]
        #         cv2.line(img_draw_1, (x, y), (w, h), (0, 255, 0), 2)
        #     cv2.imshow('img_draw_1', img_draw_1)
        #     cv2.waitKey()
        #     cv2.destroyAllWindows()
        #     angles = []
        #     for line in lines_1:
        #         x1, y1, x2, y2 = line[0]
        #         angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        #         angles.append(angle)
        #     # Tính góc nghiêng trung bình
        #     angle = np.mean(angles)
        #     if np.abs(angle) < 20:
        #         # Xoay ảnh lại để chữ số thẳng đứng
        #         height, width = image_original.shape[:2]
        #         center = (width // 2, height // 2)
        #         M = cv2.getRotationMatrix2D(center, angle, 1)
        #         img_rotated = cv2.warpAffine(image_original, M, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
        #         cv2.imshow('img_rotated', img_rotated)
        #         cv2.waitKey()
        #         cv2.destroyAllWindows()
        # else:
        #     print(f"Không tìm thấy đường thẳng trong ảnh {img_path}")
    except Exception as e:
        print(e)